# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

**План работы:**

[1  Подготовка](#section1)

[2  Обучение](#section2)

- [LogisticRegression](#section2.1)

- [DecisionTreeClassifier](#section2.2)

- [RandomForestClassifier](#section2.3)

- [LGBMClassifier](#section2.4)

- [XGBClassifier](#section2.5)

[3  Вывод](#section3)



<a id='section1'></a>
## Подготовка

In [1]:
# загружаем необходимые библиотеки
import pandas as pd
import matplotlib.pyplot as plt 
plt.style.use('seaborn-pastel')
import seaborn as sns 
import numpy as np 

from time import time
from tqdm import tqdm

pd.options.display.max_columns = None 

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords as nltk_stopwords
from nltk.corpus import wordnet
from nltk.tokenize import word_tokenize

import re

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import lightgbm as lgb
import xgboost as xgb

import warnings
warnings.filterwarnings('ignore')

# введём константу
RANDOM_STATE = 12345

Добавим необходимые файлы для английской библиотеки лемматизатора

In [2]:
nltk.download('punkt')
nltk.download('wordnet')

stopwords = set(nltk_stopwords.words('english'))

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


Загрузим данные, выведем первые 10 строк

In [3]:
df = pd.read_csv('/datasets/toxic_comments.csv')
df.head(10)

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0
5,5,"""\n\nCongratulations from me as well, use the ...",0
6,6,COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK,1
7,7,Your vandalism to the Matt Shirvington article...,0
8,8,Sorry if the word 'nonsense' was offensive to ...,0
9,9,alignment on this subject and which are contra...,0


In [4]:
# избавимся от столбца "Unnamed: 0" - он не понадобится для дальнейшей работы
df = df.drop('Unnamed: 0', axis=1)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


Пропусков нет

In [6]:
# посмотрим соотношение токсичных комментариев к общему числу
df['toxic'].value_counts()

0    143106
1     16186
Name: toxic, dtype: int64

Имеется яный дисбаланс

- От лишних символов текст очистят регулярные выражения. Для работы с регулярными выражениями в Python будем использовать встроенный модуль re. 
- Создадим функцию, которая очистит текст для будущей лемматизации.

In [7]:
def clear_text(text):
     # преобразовываем и переводим в нижний регистр
    text = re.sub(r'[^a-zA-Z ]', ' ', text.lower())
    retext = text.split() 
    text = " ".join(retext)
    return text

Перед извлечением признаков текста необходимо его упростить. 
Этапами предобработки будет следующее:
- разбиение текста на отдельные слова, фразы, символы - токенизация
- приведение слова к начальной форме - лемматизация

In [8]:
# функцию для тоекнизации и лемматизации текста
def lemmetize(words):
    lemmatizer = WordNetLemmatizer() 
    text = [] # создаём список для заполнения
    tokens = word_tokenize(words) # токенизируем текст
    for token in tokens: # лемматизируем каждый токен
        lemmetized_word = lemmatizer.lemmatize(token) 
        text.append(lemmetized_word)
    sentence = " ".join(text) # объединение в текст обратно
    return sentence

Применяем функции

In [9]:
df['lemm_text'] = df['text'].apply(clear_text)
df['lemm_text'] = df['lemm_text'].apply(lemmetize)

df['lemm_text']

0         explanation why the edits made under my userna...
1         d aww he match this background colour i m seem...
2         hey man i m really not trying to edit war it s...
3         more i can t make any real suggestion on impro...
4         you sir are my hero any chance you remember wh...
                                ...                        
159287    and for the second time of asking when your vi...
159288    you should be ashamed of yourself that is a ho...
159289    spitzer umm there no actual article for prosti...
159290    and it look like it wa actually you who put on...
159291    and i really don t think you understand i came...
Name: lemm_text, Length: 159292, dtype: object

Для определения тематики и тональности текста, алгоритм необходимо обучить на корпусе, что является набором текстов, в котором эмоции и ключевые слова уже размечены. 

Определяем тренировочную и тестовую выборки

In [10]:
train_features, test_features, train_target, test_target = train_test_split(
    df.drop('toxic', axis = 1),
    df['toxic'],
    test_size = 0.25,
    random_state = RANDOM_STATE,
     # стратифицируем текст, чтобы выборки были более сбалансированы
    stratify = df['toxic'])

In [11]:
# вытаскиваем корпусы
corpus_train = train_features['lemm_text']
corpus_test = test_features['lemm_text']
corpus_train

129315    accusation of vandalism what wa the cause and ...
8868      shorter oxford english dictionary arrived toda...
75827     kim stop you are not helping here this is fann...
137701    my blocking by user jayron i believe that this...
28407     simple quote don t you think this essentially ...
                                ...                        
54715     indeed bigdunc that page rightly say that para...
85775     it s a criticism moron it doesn t need to be s...
71196     forgive my cruddy formattingi m still relative...
55751     talk grasshopper scout i moved your comment fr...
11543                                  oppose wp commonname
Name: lemm_text, Length: 119469, dtype: object

Воспользуемся TfidfVectorizer, чтобы почистить мешок слов, добавим в него стоп-слова

In [12]:
# подгружаем счетчик и задаём стоп-слова
count_tf_idf = TfidfVectorizer(stop_words = stopwords)
# обучаем и трансформируем
tf_idf_train = count_tf_idf.fit_transform(corpus_train) 
# трансформируем тестовую без обучения
tf_idf_test = count_tf_idf.transform(corpus_test) 

print("Размер матрицы:", tf_idf_train.shape)
print("Размер матрицы:", tf_idf_test.shape)

Размер матрицы: (119469, 133602)
Размер матрицы: (39823, 133602)


**Вывод:**

- Обработка данных выполнена (токенизация, лемматизация)
- Тестовую и тренировочную выборки определены для обучения
- TF-IDF подсчитано (оценка важности слова)

<a id='section2'></a>
## Обучение

При обучении будут спользованы следующие модели: 
- LogisticRegression
- DecisionTreeClassifier
- RandomForestClassifier
- LGBMClassifier
- XGBClassifier

Создадим функцию, которая обучит и вернёт модель, создаст и заполнит таблицу для дальнейшего анализа получившихся метрик

In [13]:
# таблица для анализа
analisys = pd.DataFrame({'model':[], 'F1_model':[], 'F1_on_train':[]})
all_models = []

# функции для подбора параметров
def train_model(model, parameters):
    
    model_random = RandomizedSearchCV(
        estimator = model,
        param_distributions = parameters,
        scoring = 'f1', 
        n_jobs = -1,
        cv = 4, 
        verbose = 2
    )
    
    # подсчет скорости вычисления
    start = time()
    model_random.fit(tf_idf_train, train_target)
    print('Время подбора параметров %.2f секунд' %(time() - start))
    
    # подсчет метрик
    f1 = model_random.best_score_
    f1_on_train = f1_score(train_target, model_random.predict(tf_idf_train))
    
    print('Лучшие параметры:', model_random.best_params_)
    print('F1 обученной модели:', f1)
    print('F1 на тренировочной выборке:', f1_on_train)

    # заполним все таблицы    
    all_models.append(model_random)
    row = []
    row.extend([model, f1, f1_on_train])
    analisys.loc[len(analisys.index)] = row
    
    return model_random

<a id='section2.1'></a>
**LogisticRegression**

In [14]:
# задаём рандомные параметры
ran_lr = {"penalty": ['l1', 'l2', 'elasticnet', 'none'],
    "class_weight": ['balanced', 'none'],}

logr  = LogisticRegression(max_iter = 1000)

# обучение и сохранение значений
lr_random = train_model(logr , ran_lr)

Fitting 4 folds for each of 8 candidates, totalling 32 fits
[CV] END ..................class_weight=balanced, penalty=l1; total time=   0.0s
[CV] END ..................class_weight=balanced, penalty=l1; total time=   0.0s
[CV] END ..................class_weight=balanced, penalty=l1; total time=   0.0s
[CV] END ..................class_weight=balanced, penalty=l1; total time=   0.0s
[CV] END ..................class_weight=balanced, penalty=l2; total time=  45.4s
[CV] END ..................class_weight=balanced, penalty=l2; total time=  39.2s
[CV] END ..................class_weight=balanced, penalty=l2; total time=  57.3s
[CV] END ..................class_weight=balanced, penalty=l2; total time=  57.8s
[CV] END ..........class_weight=balanced, penalty=elasticnet; total time=   0.0s
[CV] END ..........class_weight=balanced, penalty=elasticnet; total time=   0.0s
[CV] END ..........class_weight=balanced, penalty=elasticnet; total time=   0.0s
[CV] END ..........class_weight=balanced, penalty

<a id='section2.2'></a>
**DecisionTreeClassifier**

In [15]:
ran_grid_tree = {"max_depth": list(range(45, 56))}

dtr = DecisionTreeClassifier()

dtr_random = train_model(dtr, ran_grid_tree)

Fitting 4 folds for each of 10 candidates, totalling 40 fits
[CV] END .......................................max_depth=48; total time=  25.5s
[CV] END .......................................max_depth=48; total time=  25.6s
[CV] END .......................................max_depth=48; total time=  25.5s
[CV] END .......................................max_depth=48; total time=  25.2s
[CV] END .......................................max_depth=54; total time=  28.6s
[CV] END .......................................max_depth=54; total time=  27.5s
[CV] END .......................................max_depth=54; total time=  27.9s
[CV] END .......................................max_depth=54; total time=  27.4s
[CV] END .......................................max_depth=50; total time=  26.3s
[CV] END .......................................max_depth=50; total time=  26.1s
[CV] END .......................................max_depth=50; total time=  25.8s
[CV] END .......................................

<a id='section2.3'></a>
**RandomForestClassifier**

In [16]:
ran_grid_forest = {'max_depth': [300, 310],
    'n_estimators': [12, 14],}

rfc = RandomForestClassifier(n_jobs=-1)

rfc_random = train_model(rfc, ran_grid_forest)

Fitting 4 folds for each of 4 candidates, totalling 16 fits
[CV] END .....................max_depth=300, n_estimators=12; total time= 1.0min
[CV] END .....................max_depth=300, n_estimators=12; total time= 1.0min
[CV] END .....................max_depth=300, n_estimators=12; total time= 1.0min
[CV] END .....................max_depth=300, n_estimators=12; total time= 1.0min
[CV] END .....................max_depth=300, n_estimators=14; total time= 1.2min
[CV] END .....................max_depth=300, n_estimators=14; total time= 1.2min
[CV] END .....................max_depth=300, n_estimators=14; total time= 1.1min
[CV] END .....................max_depth=300, n_estimators=14; total time= 1.1min
[CV] END .....................max_depth=310, n_estimators=12; total time= 1.0min
[CV] END .....................max_depth=310, n_estimators=12; total time= 1.0min
[CV] END .....................max_depth=310, n_estimators=12; total time= 1.0min
[CV] END .....................max_depth=310, n_es

<a id='section2.4'></a>
**LGBMClassifier**

In [17]:
rand_lgbm_param = {
    'max_depth': [15, 25],
    'learning_rate': [0.1, 0.3]
}

gbm = lgb.LGBMClassifier(
    boosting_type='gbdt',
    n_jobs=-1
)

gbm_random= train_model(gbm, rand_lgbm_param)

Fitting 4 folds for each of 4 candidates, totalling 16 fits
[CV] END ....................learning_rate=0.1, max_depth=15; total time= 2.2min
[CV] END ....................learning_rate=0.1, max_depth=15; total time= 2.2min
[CV] END ....................learning_rate=0.1, max_depth=15; total time= 2.3min
[CV] END ....................learning_rate=0.1, max_depth=15; total time= 2.3min
[CV] END ....................learning_rate=0.1, max_depth=25; total time= 2.4min
[CV] END ....................learning_rate=0.1, max_depth=25; total time= 2.5min
[CV] END ....................learning_rate=0.1, max_depth=25; total time= 2.5min
[CV] END ....................learning_rate=0.1, max_depth=25; total time= 2.4min
[CV] END ....................learning_rate=0.3, max_depth=15; total time= 2.1min
[CV] END ....................learning_rate=0.3, max_depth=15; total time= 2.1min
[CV] END ....................learning_rate=0.3, max_depth=15; total time= 2.0min
[CV] END ....................learning_rate=0.3, m

<a id='section2.5'></a>
**XGBClassifier**

In [20]:
rand_xgb_param = {
    'max_depth': [6, 7, 8, 9],
    'learning_rate': [0.5, 1.0]
}

xb = xgb.XGBClassifier(booster = 'gbtree', 
                      use_rmm = True,
                      n_jobs = -1)

xb_random = train_model(xb, rand_xgb_param)

Fitting 4 folds for each of 8 candidates, totalling 32 fits
[02:05:47] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] END .....................learning_rate=0.5, max_depth=6; total time= 2.6min
[02:08:24] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] END .....................learning_rate=0.5, max_depth=6; total time= 2.6min
[02:11:01] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[CV] END .............

**Вывод:**

- Обучены модели LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, LGBMClassifier, XGBClassifier.
- Гиперпараметры найдены.

## Выводы

Соберем в таблицу результаты обученных моделей и посмотрим результат.

In [22]:
all_names = pd.DataFrame({'names':[ 'LogisticRegression', 'DecisionTree', 'RandomForest', 'LightGBM', 'XGBoost']})
analisys = pd.concat([analisys, all_names], axis=1, join='inner')
display(analisys)

,model,F1_model,F1_on_train,names
0,LogisticRegression(max_iter=1000),0.744884,0.831681,LogisticRegression
1,DecisionTreeClassifier(),0.708474,0.823336,DecisionTree
2,RandomForestClassifier(n_jobs=-1),0.627737,0.916213,RandomForest
3,LGBMClassifier(),0.765033,0.856844,LightGBM
4,"XGBClassifier(base_score=None, booster='gbtree...",0.751659,0.856395,XGBoost


Исходя из полученных метрик качества моделей, лучшая модель на RandomizedSearchCV - LightGBM c параметрами max_depth: 25, learning_rate: 0.3. На тренировочной выборке, лучшую метрику показывает модель Случайного леса, но и худшую на подборе параметров, то есть модель переобучена и не показывает нужных метрик.

Проверим качество лучшей модели на тестовой выборке, которая уже обучена.

In [26]:
predicted = gbm_random.predict(tf_idf_test)
print('F1 :', f1_score(test_target, predicted))

F1 : 0.7680534298038124


**Вывод:**

Необходимые метрики достигнуты, модель LightGBM, обученная через RandomizedSearchCV, предсказывает с необходимой метрикой: F1 > 0.75.

**Общий вывод:**
- Данные загрежены, обработаны, проведене токенизация и лемматизация.
- Определены тестовая и тренеровачная выборки.
- TF-IDF подсчитано.
- Обучены модели LogisticRegression, DecisionTreeClassifier, RandomForestClassifier, LGBMClassifier, XGBClassifier, и найдены сответствующие гиперпараметры.
- Модель LightGBM показала значение метрики качества F1 не меньше 0.75 по условию и равно 0.7680 соответственно.